# Trabalho 2

## 4

Neste notebook estão os códigos os códigos dos 3 algoritmos num modo que permite fazer ciclo nas múltiplas divisões Treino-Teste dos dados e aplicar os 3 algoritmos par assim recolher amostras emparelhadas das métricas de cada um dos modelos.
O pedido é para os 2 melhores resultados, no entanto na altura em que esta parte do código foi realizada ainda não tinhamos a certeza de quais iam ser. Pelo que para prevenir fez-se dos 3.

### Imports

In [34]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
import statistics
import utilidades as ut

### Inicializações e variáveis

In [2]:
warnings.filterwarnings("ignore")  # Desabilitar warnings.
plt.style.use(
    "style/estilo.mplstyle")  # Garantir que se utiliza um estilo definido centralmente e comum a todos os gráficos.
%matplotlib inline

label_encoder = LabelEncoder()

ficheiro = "dados_preparados.csv"
colunas_numericas = ["Idade", "FCV", "NRP", "CA", "FAF", "TUDE", "IMC"]
colunas_classes = ["Genero", "Historico_obesidade_familiar", "FCCAC", "Fumador", "MCC", "CCER", "CBA", "TRANS"]
colunas_classes_binarias = ['Genero', 'Historico_obesidade_familiar', 'FCCAC', 'Fumador', 'MCC']
colunas_classes_multiplos = ["CCER", "CBA", "TRANS", "Label"]

## Leitura dos dados preparados

In [3]:
dados_trabalho = pd.read_csv(ficheiro)

In [4]:
dados_trabalho

,Genero,Idade,Historico_obesidade_familiar,FCCAC,FCV,NRP,CCER,Fumador,CA,MCC,FAF,TUDE,CBA,TRANS,Label,IMC
0,Feminino,21.000000,Sim,Nao,2.0,3.0,Ocasionalmente,Nao,2.000000,Nao,0.000000,1.000000,Nao,Transportes_Publicos,Peso_Normal,24.386526
1,Feminino,21.000000,Sim,Nao,3.0,3.0,Ocasionalmente,Sim,3.000000,Sim,3.000000,0.000000,Ocasionalmente,Transportes_Publicos,Peso_Normal,24.238227
2,Masculino,23.000000,Sim,Nao,2.0,3.0,Ocasionalmente,Nao,2.000000,Nao,2.000000,1.000000,Frequentemente,Transportes_Publicos,Peso_Normal,23.765432
3,Masculino,27.000000,Nao,Nao,3.0,3.0,Ocasionalmente,Nao,2.000000,Nao,2.000000,0.000000,Frequentemente,Caminhada,Excesso_Peso_Grau_I,26.851852
4,Masculino,22.000000,Nao,Nao,2.0,1.0,Ocasionalmente,Nao,2.000000,Nao,0.000000,0.000000,Ocasionalmente,Transportes_Publicos,Excesso_Peso_Grau_II,28.342381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Feminino,20.976842,Sim,Sim,3.0,3.0,Ocasionalmente,Nao,1.728139,Nao,1.676269,0.906247,Ocasionalmente,Transportes_Publicos,Obesidade_Mórbida,44.901475
2107,Feminino,21.982942,Sim,Sim,3.0,3.0,Ocasionalmente,Nao,2.005130,Nao,1.341390,0.599270,Ocasionalmente,Transportes_Publicos,Obesidade_Mórbida,43.741923
2108,Feminino,22.524036,Sim,Sim,3.0,3.0,Ocasionalmente,Nao,2.054193,Nao,1.414209,0.646288,Ocasionalmente,Transportes_Publicos,Obesidade_Mórbida,43.543817
2109,Feminino,24.361936,Sim,Sim,3.0,3.0,Ocasionalmente,Nao,2.852339,Nao,1.139107,0.586035,Ocasionalmente,Transportes_Publicos,Obesidade_Mórbida,44.071535


Temos no entanto de realizar primeiro o encoding das classes para valores numéricos, esta operação é realizada usando o ``sklearn.preprocessing.LabelEncoder`` em todas as colunas com apenas 2 classes.

In [5]:
ut.titulo("Valores codificados por atributo")

for coluna in colunas_classes:
    if dados_trabalho[coluna].dtype == 'object':
        dados_trabalho[coluna] = label_encoder.fit_transform(dados_trabalho[coluna].values)
        ut.etiqueta_e_valor(coluna, str(sorted(dados_trabalho[coluna].unique())))

Valores codificados por atributo
Genero: [0, 1]
Historico_obesidade_familiar: [0, 1]
FCCAC: [0, 1]
Fumador: [0, 1]
MCC: [0, 1]
CCER: [0, 1, 2, 3]
CBA: [0, 1, 2, 3]
TRANS: [0, 1, 2, 3, 4]


A coluna *Label* é uma "classificação", não um preditor. Tem de ser removida.

In [6]:
dados_trabalho.drop("Label", axis=1, inplace=True)

| Target | Preditores                   |
|--------|------------------------------|
| IMC    | Ver lista abaixo (X.columns) |

O target não muda

In [7]:
y = dados_trabalho.IMC

## Preditores

Colunas originais, antes do get_dummies:
lista_preditores = ["Idade", "Genero", "Historico_obesidade_familiar", "FCCAC", "FCV", "NRP", "CCER", "Fumador", "CA", "MCC", "FAF", "TUDE", "CBA", "TRANS"]

Obter a lista de todos os atributos (preditores) do dataset, é preciso remover a coluna "IMC"

In [8]:
X = dados_trabalho.drop("IMC", axis=1)
X.columns

Index(['Genero', 'Idade', 'Historico_obesidade_familiar', 'FCCAC', 'FCV',
       'NRP', 'CCER', 'Fumador', 'CA', 'MCC', 'FAF', 'TUDE', 'CBA', 'TRANS'],
      dtype='object')

##  Cálculo

A comparação tem de ser feita num ciclo em que cada algoritmo irá operar sobre o mesmo conjunto de treino e teste, que muda a cada ciclo. 

### Função de cálculo para 1 execução do LinearRegression

In [9]:
def linear_regression(X_treino, X_teste, y_treino, y_teste):
    mlr = LinearRegression()
    mlr.fit(X_treino, y_treino)
    y_pred_teste = mlr.predict(X_teste)
    # Previsão
    y_pred_treino = mlr.predict(X_treino)  # Teste
    y_pred_teste = mlr.predict(X_teste)  # Teste
    # Métricas
    mae_treino = round(mean_absolute_error(y_treino, y_pred_treino), 4)
    mse_treino = round(mean_squared_error(y_treino, y_pred_treino), 4)
    rmse_treino = round(np.sqrt(mean_squared_error(y_treino, y_pred_treino)), 4)
    mae_teste = round(mean_absolute_error(y_teste, y_pred_teste), 4)
    mse_teste = round(mean_squared_error(y_teste, y_pred_teste), 4)
    rmse_teste = round(np.sqrt(mean_squared_error(y_teste, y_pred_teste)), 4)
    r_squared = mlr.score(X, y) * 100
    ut.etiqueta_e_valor(
        f"LinearRegression random_state={idx}",
        f"TREINO(MAE={mae_treino} / MSE={mse_treino} / RMSE={rmse_treino}) / "
        f"TESTE(MAE={mae_teste} / MSE={mse_teste} / RMSE={rmse_teste}) / "
        f"R_squared={r_squared:.5}"
    )
    return "MLPRegressor", idx, mae_treino, mse_treino, rmse_treino, mae_teste, mse_teste, rmse_teste, r_squared    

### Função para cálculo de 1 execução de DecisionTreeRegression

In [10]:
def decision_tree_regressor(X_treino, X_teste, y_treino, y_teste):
    arvore_decisao = DecisionTreeRegressor(
        # random_state=42,
        max_depth=6,
        min_samples_split=3
    )
    _ = arvore_decisao.fit(X_treino, y_treino)
    y_pred_treino = arvore_decisao.predict(X_treino)
    y_pred_teste = arvore_decisao.predict(X_teste)
    # Métricas
    mae_treino = round(mean_absolute_error(y_treino, y_pred_treino), 4)
    mse_treino = round(mean_squared_error(y_treino, y_pred_treino), 4)
    rmse_treino = round(np.sqrt(mean_squared_error(y_treino, y_pred_treino)), 4)
    mae_teste = round(mean_absolute_error(y_teste, y_pred_teste), 4)
    mse_teste = round(mean_squared_error(y_teste, y_pred_teste), 4)
    rmse_teste = round(np.sqrt(mean_squared_error(y_teste, y_pred_teste)), 4)
    r_squared = arvore_decisao.score(X, y) * 100
    ut.etiqueta_e_valor(
        f"LinearRegression random_state={idx}",
        f"TREINO(MAE={mae_treino} / MSE={mse_treino} / RMSE={rmse_treino}) / "
        f"TESTE(MAE={mae_teste} / MSE={mse_teste} / RMSE={rmse_teste}) / "
        f"R_squared={r_squared:.5}"
    )
    return "DecisionTreeRegressor", idx, mae_treino, mse_treino, rmse_treino, mae_teste, mse_teste, rmse_teste, r_squared

### Função para cálculo de 1 execução do MLPRegressor

In [11]:
def mlpregressor(X_treino, X_teste, y_treino, y_teste, layers, activation, solver):
    modelo_nn = MLPRegressor(
        hidden_layer_sizes=layers,
        activation=activation,
        solver=solver,
        max_iter=1000,
        learning_rate_init=0.001
    )
    modelo_nn.fit(X_treino, y_treino)  # Treino
    # Previsão
    y_pred_treino = modelo_nn.predict(X_treino)  # Teste
    y_pred_teste = modelo_nn.predict(X_teste)  # Teste
    # Métricas
    mae_treino = round(mean_absolute_error(y_treino, y_pred_treino), 4)
    mse_treino = round(mean_squared_error(y_treino, y_pred_treino), 4)
    rmse_treino = round(np.sqrt(mean_squared_error(y_treino, y_pred_treino)), 4)
    mae_teste = round(mean_absolute_error(y_teste, y_pred_teste), 4)
    mse_teste = round(mean_squared_error(y_teste, y_pred_teste), 4)
    rmse_teste = round(np.sqrt(mean_squared_error(y_teste, y_pred_teste)), 4)
    r_squared = modelo_nn.score(X, y) * 100
    ut.etiqueta_e_valor(
        f"LinearRegression random_state={idx}",
        f"TREINO(MAE={mae_treino} / MSE={mse_treino} / RMSE={rmse_treino}) / "
        f"TESTE(MAE={mae_teste} / MSE={mse_teste} / RMSE={rmse_teste}) / "
        f"R_squared={r_squared:.5}"
    )
    return "MLPRegressor", idx, mae_treino, mse_treino, rmse_treino, mae_teste, mse_teste, rmse_teste, r_squared    

### Ciclo para obter os vários valores

In [12]:
resultados_mlr = []
resultados_dtr = []
resultados_mlpr = []

for idx in range(0, 100, 2):
    # Criação de novos grupos de treino e de teste. 
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=idx)
    # Invocação dos algoritmos
    resultados_mlr.append(linear_regression(X_train, X_test, y_train, y_test))
    resultados_dtr.append(decision_tree_regressor(X_train, X_test, y_train, y_test))
    resultados_mlpr.append(mlpregressor(X_train, X_test, y_train, y_test, [5, 6, 5], "tanh", "lbfgs"))

LinearRegression random_state=0: TREINO(MAE=4.5642 / MSE=32.7556 / RMSE=5.7233) / TESTE(MAE=4.9737 / MSE=36.2051 / RMSE=6.0171) / R_squared=47.862
LinearRegression random_state=0: TREINO(MAE=2.4819 / MSE=11.4803 / RMSE=3.3883) / TESTE(MAE=2.8046 / MSE=15.4927 / RMSE=3.9361) / R_squared=80.851
LinearRegression random_state=0: TREINO(MAE=6.5922 / MSE=62.6148 / RMSE=7.913) / TESTE(MAE=7.0904 / MSE=70.2956 / RMSE=8.3842) / R_squared=-0.0043222
LinearRegression random_state=2: TREINO(MAE=4.6847 / MSE=33.5782 / RMSE=5.7947) / TESTE(MAE=4.5025 / MSE=32.9875 / RMSE=5.7435) / R_squared=47.842
LinearRegression random_state=2: TREINO(MAE=2.3855 / MSE=11.0174 / RMSE=3.3192) / TESTE(MAE=3.0292 / MSE=18.5503 / RMSE=4.307) / R_squared=80.473
LinearRegression random_state=2: TREINO(MAE=5.7513 / MSE=49.8662 / RMSE=7.0616) / TESTE(MAE=5.8753 / MSE=52.7342 / RMSE=7.2618) / R_squared=21.372
LinearRegression random_state=4: TREINO(MAE=4.6012 / MSE=32.9281 / RMSE=5.7383) / TESTE(MAE=4.7287 / MSE=35.6855 / R

## Teste

As amostras são emparelhadas, uma vez que foram testes realizados sobre o mesmo conjunto de dados de treino e de teste. Não havendo assim necessidade de testar a variância das mesmas.

Recolher os valores dos erros das listas de resultados

In [18]:
rmse_mlr = [metricas[8] for metricas in resultados_mlr]
rmse_dtr = [metricas[8] for metricas in resultados_dtr]
rmse_mlpr = [metricas[8] for metricas in resultados_mlpr]

mse_mlr = [metricas[7] for metricas in resultados_mlr]
mse_dtr = [metricas[7] for metricas in resultados_dtr]
mse_mlpr = [metricas[7] for metricas in resultados_mlpr]

mae_mlr = [metricas[6] for metricas in resultados_mlr]
mae_dtr = [metricas[6] for metricas in resultados_dtr]
mae_mlpr = [metricas[6] for metricas in resultados_mlpr]

Imprimir os melhores resultados, que no caso da regressão são os menores erros.

In [19]:
ut.titulo("RMSE")
ut.etiqueta_e_valor("MLR", min(rmse_mlr))
ut.etiqueta_e_valor("DTR", min(rmse_dtr))
ut.etiqueta_e_valor("MLPR", min(rmse_mlpr))

ut.titulo("MSE")
ut.etiqueta_e_valor("MLR", min(mse_mlr))
ut.etiqueta_e_valor("DTR", min(mse_dtr))
ut.etiqueta_e_valor("MLPR", min(mse_mlpr))

ut.titulo("MAE")
ut.etiqueta_e_valor("MLR", min(mae_mlr))
ut.etiqueta_e_valor("DTR", min(mae_dtr))
ut.etiqueta_e_valor("MLPR", min(mae_mlpr))

RMSE
MLR: 47.727123185057685
DTR: 76.20920092335733
MLPR: -0.06389278539464804
MSE
MLR: 5.381
DTR: 3.6379
MLPR: 4.0109
MAE
MLR: 28.9556
DTR: 13.2347
MLPR: 16.0873


Face aos dados, os melhores parecem ser o Regressão Linear Múltipla e a rede neural MLPRegressor.

#### Hipotese

$$H_0: \bar X_{MLR} \geq \bar X_{MLPR} \quad Vs \quad H_1: \bar X_{MLR} < \bar X_{MLPR}$$

Utilizaçao de *t-student* à média, para amostras emparelhadas.

In [37]:
teste_mae = stats.ttest_rel(mae_mlr, mae_mlpr, alternative="less")
ut.texto(f"O valor de pvalue é {teste_mae.pvalue:.4f}")

O valor de pvalue é 0.0000


In [38]:
teste_mse = stats.ttest_rel(mse_mlr, mse_mlpr, alternative="less")
ut.texto(f"O valor de pvalue é {teste_mse.pvalue:.4f}")

O valor de pvalue é 0.0000


In [39]:
teste_rmse = stats.ttest_rel(rmse_mlr, rmse_mlpr, alternative="less")
ut.texto(f"O valor de pvalue é {teste_rmse.pvalue:.4f}")

O valor de pvalue é 1.0000
